# Clasificador de Ítems en Categorías

Notebook creado para realizar una clasificación o agrupación de los ítems detallados de los presupuestos de laboratorios de transformación cultural. Se utiliza la API de la IA Gemini de Google, se realiza el procesamiento por lotes de archivos, con 200 registros cada uno.

- Realizado por: Javier Mauricio Ojeda Pepinosa
- E-mail: javier.ojeda@scrd.gov.co
- Fecha: 25/07/2025
- Dirección Observatorio y Gestión del Conocimiento Cultural

In [1]:
import requests
import json
import os
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
import time
from tqdm.auto import tqdm

In [2]:
MODEL_ID = "gemini-2.0-flash-lite"
GENERATE_CONTENT_API = "generateContent" # Cambiado a 'generateContent' para no usar stream

load_dotenv() # Carga las variables del archivo .env
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

API_ENDPOINT = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL_ID}:{GENERATE_CONTENT_API}"

# Cabeceras de la solicitud
headers = {
    'Content-Type': 'application/json'
}

batch_responses_folder = 'clasificados'
batch_folder = 'parts'

In [3]:
# Instrucción general del prompt, lo que debe hacer con los datos
system_instruction = """
Necesito clasificar ítems de compras para eventos y talleres en alguna de estas categorías:
---
1. Ferretería
Ferretería (dispositivos escenográficos, embellecimiento, urbanismo táctico).
2. Difusión y material POP
Difusión y material POP (flyers, creadores de contenido, esferos, tulas, etc).
3. Logística
Incluye: refrigerios, lugares, materiales de papelería, pólizas, SUGA, seguridad, escenografía, escenarios, equipo de audio, 
micrófonos, consolas, tarimas, pantallas, monitores, parlantes, techos, cabinas line
4. Artistas
Músicos, actores, presentación de artistas en general, intervenciones artísticas
5. Desarrollos digitales 
(micrositios, apps, etc).
6. Accesorios y prendas
Cosas e insumos como aretes, topos, pulseras, anillos, ropa, collares, bolsos
7. Servicios personales
Servicios prestados por personas que no se identifican explícitamente como artistas, normalmente son nombres de personas.
9. Otro
Para los que no corresponden a ninguna de las categorías anteriores.
---
## Entrada:
Te voy a dar un listado de hasta 200 ítems, con un formato como el de CSV con 2 columnas: id (numérico) y descripción (texto). Ejemplo
158,Vinilo tipo 1 alta cobertura Tito Pabòn color azul turquesa x caneca
159,Vinilo tipo 1 alta cobertura Tito Pabòn color rojo x caneca
160,Vinilo tipo 1 alta cobertura Tito Pabòn color amarillo x caneca
## Salida esperada:
Espero recibir una tabla para poderla copiar en el navegador y pegar en Excel, con las columnas id y categoria, separadas por el símbolo
"""

In [4]:
# Lista de archivos (lotes) a procesar
parts_dir = Path.cwd() / "parts"
batch_files = sorted([f.name for f in parts_dir.iterdir() if f.is_file()])

# batch_files  # la celda mostrará la lista de nombres, p.ej. ['lab1_01.csv', 'lab1_02.csv', ...]

In [ ]:
# Lee el contenido del archivo que se va a procesar
def get_request_text(request_text_file_path):
    with open(request_text_file_path, "r", encoding="utf-8") as f:
        request_text = f.read()
    return request_text

In [ ]:
# Devuelve el array con los contenidos de la conversación y petición con el modelo LLM
def get_contents(request_text):
    contents = [
        {
            "role": "user",
            "parts": [
                {"text": "id,DESCRIPCIÓN\n429,Suministro e instalacion de 58 ladrillo tipo adoquin\n430,Limpieza y desmanchado de muros en ladrillo a la vista por grafitti 135 mts 2\n431,Reparacion de canecas basura por deterioro\n432,Suministro pintura e instalacion de piezas de madera para sillas exteriores\n433,Suministro y aplicacion de anticorrosivo y pintura tipo esmalte color negro sobre piezas metalicas\u00a0 \u00a0y sellante sobre piezas de madera\n434,Mantenimiento parque infantil\n435,Reparación / Arreglo mediante fundición de concreto de silla a piso (demoler adoquin y fundir dados)\n436,Suministro y aplicación de pintura Koraza (alta intemperie) en muros y ladrillos a 3 manos 135 metros\n2449,\"Presentación en vivo de la agrupación Blue train, backline sonido y luces para el show\"\n2450,\"The keens: Servicio de presentación musical de 2 horas de duración total dividido en 4 segmentos, realizado por banda de 2 integrantes, incluyendo instrumentos musicales y percusión, micrófonos, equipos de sonido (pedales, cables, consola), transporte de equipos, entro otros costos asociados. A realizarse en el Centro Cultural A Fuego, el 13 de diciembre de 2024.\"\n2451,Concierto con el dúo Carlos Reyes y Germán Pinilla en Morrison Club DC el día viernes 13 de diciembre de 2024\n2452,\"Presentación musical: dos sets de una hora, para un total de 120 minutos de concierto de la agrupación RAFA PINO & EL MACUARE.\"\n2453,\"Presentación musical: dos sets de una hora, para un total de 120 minutos de concierto de la agrupación RAFA PINO (La Tarta: Domingo 15 de diciembre de 2024)\"\n2454,Motherfunkers: Presentación de show musical Los Ajisocios el día sábado 14 de diciembre del 2024 en el restaurante Canasto.\n2455,Show en vivo en el establecimiento la rosconeria: 2 horas de show en vivo 2 músicos + rider técnico + transporte del grupo PROYECTO INDEENDIENTE JUANFE SÁNCHEZ\n2456,\"Presentación músical de la agrupación Rey Mostaza el día 15 de diciembre del año en curso en el IRISH PUB 120 minutos de Show músical además de montaje de sonido, equipos, transporte y logistica\"\n2457,Presentación agrupación Dimara el día 15 de diciembre de 2024\n2458,Presentación musical dos sets de una hora para un total de 120 minutos de concierto de la agrupación El Macuare\n2459,Felipe Andrés García Millán: Presentación viernes 14 de diciembre una hora de música en vivo\n2460,2 Horas De Show En Vivo (Proyecto INDEENDIENTE JUANFE Sánchez) 2 Músicos\u00a0 + rider técnico + transporte del grupo\n2461,JERONIMO SANABRIA\n2462,SEBASTIAN - CAZADORESHISTORIAS\n2463,LINA LEON\n2464,JUSTTHETABLE JUSTO\n2465,NOSOMOSFOODIES\n2466,ANDREA AVILA\n2467,MODAEATS\n2468,FELIPE CALDERON\n2469,MARCELAROSA\n2470,JUAN DIEGO LO RECOMIENDO\n2471,\"Diseño, diagramación y montaje piezas gráficas para impresión y redes.\"\n2472,\"Gigantografía Largo: 2,30 metros. Ancho: 1,57 metros.\u00a0 Vertical.\u00a0 imrpesión y estructura tubular en aluminio (incluye transporte)\"\n2473,\"Letras: USAQUÉN Dim gral: 5.5 m ancho x 1,2 m alto x 15 cm prof (entrega pendiente)\"\n2474,\"Arco Ancho: 4 metros Alto: 2,50 metros (elaborado pendiente por instalar)\"\n2475,Afiches para residentes Pliego Polipropileno - 200 gr. Vinilo en acabado brillante.\n2476,\"Tótems de 2,00m x 0,50m Impresión en lona vinílica banner para exteriores, con estructura rígida en aluminio desarmable.\"\n2477,\"Diseño e impresiòn de banderines 100 mTS con 25cm distancia entre aristas, en lona vinílica banner para exteriores.\"\n2478,Mockups (Incluye instalación y cableado)\n2479,\"Tablero Retroiluminado con impresion banner exteriores, estructura metálica, ruedas de seguridad y bateria de alimentacion eléctrica.\"\n2716,COLLAR LARGO DE ESFERA DEGRADE (AMARILLO Y AZUL)\n2717,COLLAR EXTRA LARGO (MULTICOLOR Y ROJO)\n2718,COLLAR LARGO CON SEMILLAS\n2719,PULSERAS EN CASCA DE NARANJA\n2720,PULSERAS MULTICOLOR EN CASCA DE NARANJA\n2721,CHINCHORRO O AMACA TEJIDO\n2722,CHINCHORRO O AMACA TEJIDO\n2723,BOLSOS EN TRAPILLO\n2724,TEJIDO WAYUU\n2725,PANTUFLAS EN TRAPILLO\n2726,ORGANIZADOR TEJIDO\n2727,TOP TEJIDO\n2728,BOLSO TEJIDO PEQUEÑO\n2729,BOLSO TEJIDO\n2730,MORDELON TEJIDO PARA PERRO\n2731,ORGANIZADOR NAVIDEÑO\n2732,BOLSO TEJIDO\n2733,ORGANIZADOR TEJIDO\n2734,ORGANIZADOR TEJIDO PEQUEÑO\n2735,TOP TEJIDO\n2736,INDIVIDUALES NAVIDEÑOS\n2737,PORTA CUBIERTOS\n2738,PRENDEDORES\n2739,PESEBRES EN FOSFORO\n2740,PRENDEDORES GRANDES NAVIDEÑOS\n2741,ARETES\n2742,ARETES PEQUEÑOS\n2743,MOCHILA LANA VIRGEN\n2744,MOCHILA GROCHÉ GRANDE\n2745,MOCHILA GROCHÉ\n2746,BUFANDA LANA VIRGEN\n2747,MOCHILA COLOMBIA\n2748,BUFANDA GRIS TEJIDA\n2749,CHAL\n2750,MOCHILA PEQUEÑA\n2751,LLAVERO TEJIDO\n2752,COLLAR Y ARETES\n2753,JUEGO DE ARETES COLORES\n2754,JUEGO DE ARETES ARO\n2755,JUEGO DE ARETES FLOR\n2756,JUEGO DE ARETES PIEDRAS\n2757,JUEGO DE ARETES NAVIDEÑOS\n2758,TOPOS MADRE PERLA\n2759,DIJE LABRADORITA\n2760,LUNA CUARZO CRISTAL\n2761,TOPOS AGATA\n2762,ARETES LARGOS\n2763,ARETES LABRADORITAS\n2764,TOPOS MARIPOSA\n2765,ARETES LARGOS AGATA\n2766,ARETES HOJA GRANDE\n2767,SERPENTINA DE BRONCE BAÑADA EN ORO\n2768,CANDONGAS Y ARETES DE HOJA\n2769,ANILLO PIEDRA SOL\n2770,COLLAR CAMINO INCA\n2771,COLLAR PRECOLOMBINO\n2772,PULSERA PUNTO PERUANO\n2773,COLLAR MACRAMÉ FLOR\n2774,TEJIDO MACRAMÉ NUDO FUSIÓN\n2775,COLLAR MORRALLA\n2776,ARBOL DE LA VIDA EN BRONCE\n"}
            ]
        },
        {
            "role": "model",
            "parts": [
                {"text": "```\nid,categoria\n429,1. Ferretería\n430,1. Ferretería\n431,1. Ferretería\n432,1. Ferretería\n433,1. Ferretería\n434,1. Ferretería\n435,1. Ferretería\n436,1. Ferretería\n2449,4. Artistas\n2450,4. Artistas\n2451,4. Artistas\n2452,4. Artistas\n2453,4. Artistas\n2454,4. Artistas\n2455,4. Artistas\n2456,4. Artistas\n2457,4. Artistas\n2458,4. Artistas\n2459,4. Artistas\n2460,4. Artistas\n2461,7. Servicios personales\n2462,7. Servicios personales\n2463,7. Servicios personales\n2464,7. Servicios personales\n2465,7. Servicios personales\n2466,7. Servicios personales\n2467,7. Servicios personales\n2468,7. Servicios personales\n2469,7. Servicios personales\n2470,7. Servicios personales\n2471,2. Difusión y material POP\n2472,2. Difusión y material POP\n2473,2. Difusión y material POP\n2474,2. Difusión y material POP\n2475,2. Difusión y material POP\n2476,2. Difusión y material POP\n2477,2. Difusión y material POP\n2478,2. Difusión y material POP\n2479,2. Difusión y material POP\n2716,6. Accesorios y prendas\n2717,6. Accesorios y prendas\n2718,6. Accesorios y prendas\n2719,6. Accesorios y prendas\n2720,6. Accesorios y prendas\n2721,6. Accesorios y prendas\n2722,6. Accesorios y prendas\n2723,6. Accesorios y prendas\n2724,6. Accesorios y prendas\n2725,6. Accesorios y prendas\n2726,6. Accesorios y prendas\n2727,6. Accesorios y prendas\n2728,6. Accesorios y prendas\n2729,6. Accesorios y prendas\n2730,6. Accesorios y prendas\n2731,6. Accesorios y prendas\n2732,6. Accesorios y prendas\n2733,6. Accesorios y prendas\n2734,6. Accesorios y prendas\n2735,6. Accesorios y prendas\n2736,6. Accesorios y prendas\n2737,6. Accesorios y prendas\n2738,6. Accesorios y prendas\n2739,6. Accesorios y prendas\n2740,6. Accesorios y prendas\n2741,6. Accesorios y prendas\n2742,6. Accesorios y prendas\n2743,6. Accesorios y prendas\n2744,6. Accesorios y prendas\n2745,6. Accesorios y prendas\n2746,6. Accesorios y prendas\n2747,6. Accesorios y prendas\n2748,6. Accesorios y prendas\n2749,6. Accesorios y prendas\n2750,6. Accesorios y prendas\n2751,6. Accesorios y prendas\n2752,6. Accesorios y prendas\n2753,6. Accesorios y prendas\n2754,6. Accesorios y prendas\n2755,6. Accesorios y prendas\n2756,6. Accesorios y prendas\n2757,6. Accesorios y prendas\n2758,6. Accesorios y prendas\n2759,6. Accesorios y prendas\n2760,6. Accesorios y prendas\n2761,6. Accesorios y prendas\n2762,6. Accesorios y prendas\n2763,6. Accesorios y prendas\n2764,6. Accesorios y prendas\n2765,6. Accesorios y prendas\n2766,6. Accesorios y prendas\n2767,6. Accesorios y prendas\n2768,6. Accesorios y prendas\n2769,6. Accesorios y prendas\n2770,6. Accesorios y prendas\n2771,6. Accesorios y prendas\n2772,6. Accesorios y prendas\n2773,6. Accesorios y prendas\n2774,6. Accesorios y prendas\n2775,6. Accesorios y prendas\n2776,6. Accesorios y prendas\n```"}
            ]
        },
        {
            "role": "user",
            "parts": [
                {"text": request_text}
            ]
        },
    ]
    return contents

In [ ]:
# A partir de los contenidos y el system_instrution, devuelve el payload se se enviará
# en la petición
def get_payload(contents):
    # El cuerpo de la solicitud (payload)
    payload = {
        "contents": contents,
        "system_instruction": {
            "parts": [
                {"text": system_instruction}
            ]
        },
        "generationConfig": {
            "temperature": 0.9,
            "top_p": 1,
            "top_k": 32,
            "max_output_tokens": 8192
        },
        "safetySettings": [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"}
        ]
    }
    return payload

In [ ]:
# Ejecuta la petición a la API de gemini, request
def get_gemini_response(payload, max_retries=3, backoff=2):
    """
    Manda la petición y devuelve {'status': 1, 'text_response': '...'}
    En caso de fallo transitorio intenta reintentar hasta max_retries
    con backoff exponencial.
    """
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.post(
                API_ENDPOINT,
                params={"key": GEMINI_API_KEY},
                headers=headers,
                data=json.dumps(payload),
                timeout=30,           # evita colgarte indefinidamente
            )
            r.raise_for_status()
            full_text = "".join(
                part["text"]
                for cand in r.json().get("candidates", [])
                for part in cand.get("content", {}).get("parts", [])
                if "text" in part
            ).strip()
            if full_text.startswith("```") and full_text.endswith("```"):
                full_text = full_text[3:-3].strip()
            return {"status": 1, "text_response": full_text}

        # --- errores recuperables ---------------------------------
        except (requests.Timeout, requests.ConnectionError) as e:
            print(f"[intento {attempt}] error de red:", e)
        except requests.HTTPError as e:
            # 5xx → se puede intentar reintentar; 4xx suele ser error del cliente
            if 500 <= r.status_code < 600 and attempt < max_retries:
                print(f"[intento {attempt}] HTTP 5xx, reintentando…")
            else:
                return {"status": 0, "text_response": f"HTTP error: {e}\n{r.text}"}
        # --- otros errores ----------------------------------------
        except Exception as e:
            return {"status": 0, "text_response": f"Error inesperado: {e}"}

        # back‑off exponencial
        time.sleep(backoff ** attempt)

    return {"status": 0, "text_response": "Falló tras varios reintentos"}

In [ ]:
# Recore todos los archivos de la carpeta parts, solicita procesamiento a Gemini
# y los guarda en la carpeta de respuestas
for file_name in tqdm(batch_files, desc="Procesando lotes"):
    # 1. Construye las variables de la petición
    request_text_file_path = f'{batch_folder}/{file_name}'
    request_text = get_request_text(request_text_file_path)
    contents     = get_contents(request_text)
    payload      = get_payload(contents)

    # 2. llama a la API (bloqueante)
    gemini_response = get_gemini_response(payload)

    # 3. guarda si hubo éxito
    if gemini_response["status"] == 1:
        output_filename = f"{batch_responses_folder}/{file_name}"
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(gemini_response['text_response'])
        print(f"✔ Guardado: {output_filename}")
    else:
        print(f"✖ Falló {file_name}\n{gemini_response['text_response']}")

    # 4. (opcional) pausa breve para no saturar la cuota
    time.sleep(1)   # ajústalo a tu límite de QPS
    

Procesando lotes:   0%|          | 0/41 [00:00<?, ?it/s]

✔ Guardado: clasificados/egipto_01.csv
✔ Guardado: clasificados/egipto_02.csv
✔ Guardado: clasificados/egipto_03.csv
✔ Guardado: clasificados/estacion_museo_nacional_tm_01.csv
✔ Guardado: clasificados/estacion_museo_nacional_tm_02.csv
✔ Guardado: clasificados/estacion_museo_nacional_tm_03.csv
✔ Guardado: clasificados/estacion_polo_tm_01.csv
✔ Guardado: clasificados/estacion_polo_tm_02.csv
✔ Guardado: clasificados/gaitana_01.csv
✔ Guardado: clasificados/gaitana_02.csv
✔ Guardado: clasificados/gaitana_03.csv
✔ Guardado: clasificados/gaitana_04.csv
✔ Guardado: clasificados/gaitana_05.csv
✔ Guardado: clasificados/galerias_01.csv
✔ Guardado: clasificados/galerias_02.csv
✔ Guardado: clasificados/la_esmeralda_01.csv
✔ Guardado: clasificados/la_esmeralda_02.csv
✔ Guardado: clasificados/la_esmeralda_03.csv
✔ Guardado: clasificados/martires_01.csv
✔ Guardado: clasificados/martires_02.csv
✔ Guardado: clasificados/martires_03.csv
✔ Guardado: clasificados/martires_04.csv
✔ Guardado: clasificados/ma